# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1465]:
pip install pandas numpy matplotlib seaborn scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1466]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [1467]:
cluster_dataset = './clean/data-science-jobs-salaries-clean.csv'

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [1470]:
# **2. Memuat Dataset Hasil Clustering**
df = pd.read_csv(cluster_dataset)
print(df.head())

   work_year  salary  salary_in_usd  job_title_Data Analyst  (Remote)  \
0       2022  186597         136086                             False   
1       2020  110630          67982                             False   
2       2022   61280         153309                             False   
3       2022  154130         135242                             False   
4       2020  172312          35156                             False   

   job_title_Data Analyst  in office  job_title_Data Engineer    \
0                              False                      False   
1                              False                      False   
2                              False                      False   
3                               True                      False   
4                              False                      False   

   job_title_Data Engineer  (Remote)  job_title_Data Engineer  in office  \
0                              False                               False   
1     

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [1473]:
# Handle categorical data: encoding categorical columns
def preprocessData(df):
    for col in df.select_dtypes(include=['object']).columns:
        if col == 'cluster':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
        else:
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    return df

In [1474]:
# **3. Data Splitting**
# Membagi dataset menjadi data latih dan uji

df = preprocessData(df)

X = df.drop(columns=['Cluster'])
y = df['Cluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [1478]:
# Memeriksa data kosong pada dataset cluster
print(X_train.isnull().sum())

work_year                                         0
salary                                            0
salary_in_usd                                     0
job_title_Data Analyst  (Remote)                  0
job_title_Data Analyst  in office                 0
job_title_Data Engineer                           0
job_title_Data Engineer  (Remote)                 0
job_title_Data Engineer  in office                0
job_title_Data Scientist                          0
job_title_Data Scientist  (Remote)                0
job_title_Data Scientist  in office               0
job_title_Machine Learning Engineer               0
job_title_Machine Learning Engineer  (Remote)     0
job_title_Machine Learning Engineer  in office    0
job_title_Statistician                            0
job_title_Statistician  (Remote)                  0
job_title_Statistician  in office                 0
job_category_Data Science                         0
job_category_Engineering                          0
job_category

In [1479]:
print(y_train.isnull().sum())

0


In [1480]:
# Mendeteksi kolom kategorikal dan numerik
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(exclude=['object']).columns.tolist()

In [1481]:
# Membuat transformasi kolom kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ]
)

In [1482]:
# Membuat pipeline dengan preprocessing dan model serta melatih model menggunakan data latih
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['work_year', 'salary',
                                                   'salary_in_usd',
                                                   'job_title_Data Analyst  '
                                                   '(Remote)',
                                                   'job_title_Data Analyst  in '
                                                   'office',
                                                   'job_title_Data Engineer  ',
                                                   'job_title_Data Engineer  '
                                                   '(Remote)',
                                                   'job_title_Data Engineer  '
                                                   'in office',
                                                   'job_title_Data Scientist  ',
                                                   'job_title_Data Scientist  '
                                                   '(Remote)',...
                                                   'job_category_ML/AI',
                                                   'salary_currency_GBP',
                                                   'salary_currency_INR',
                                                   'salary_currency_JPY',
                                                   'salary_currency_us dolars',
                                                   'employee_residence_DE',
                                                   'employee_residence_IN',
                                                   'employee_residence_JP',
                                                   'employee_residence_MX',
                                                   'employee_residence_UK',
                                                   'employee_residence_US', ...]),
                                                 ('cat', OneHotEncoder(),
                                                  [])])),
                ('classifier', RandomForestClassifier(random_state=42))])

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [1486]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class F1 score
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print(f"{model.__class__.__name__}")
    print(f'Accuracy Score: {accuracy:.2f}')
    print(f'F1-Score: {f1:.2f}')
    print('Model Confusion Matrix:')
    print(conf_matrix)
    print('-' * 50)

# Initialize models
models = [
    RandomForestClassifier(),
    LogisticRegression(solver='liblinear', max_iter=1000),
    SVC()
]

# Evaluate models
for model in models:
    evaluate_model(model, X_train, X_test, y_train, y_test)

RandomForestClassifier
Accuracy Score: 1.00
F1-Score: 1.00
Model Confusion Matrix:
[[131   0   0   0   0]
 [  0 227   0   0   0]
 [  0   0 125   0   0]
 [  0   0   0 412   0]
 [  0   0   0   0 105]]
--------------------------------------------------
LogisticRegression
Accuracy Score: 0.12
F1-Score: 0.06
Model Confusion Matrix:
[[ 25   0  85   0  21]
 [ 23   0 173   0  31]
 [ 25   0  69   0  31]
 [ 43   0 303   0  66]
 [ 25   0  59   0  21]]
--------------------------------------------------
SVC
Accuracy Score: 0.12
F1-Score: 0.03
Model Confusion Matrix:
[[  0   0 123   0   8]
 [  0   0 219   0   8]
 [  0   0 118   0   7]
 [  0   0 399   0  13]
 [  0   0 102   0   3]]
--------------------------------------------------


Berikut adalah hasil evaluasi model yang telah Anda berikan dalam versi Bahasa Indonesia:

---

### **Hasil Evaluasi**

#### **Random Forest Classifier**
- **Accuracy Score**: 1.00
- **F1-Score**: 1.00
- **Model Confusion Matrix**:
  ```
  [[131   0   0   0   0]
   [  0 227   0   0   0]
   [  0   0 125   0   0]
   [  0   0   0 412   0]
   [  0   0   0   0 105]]
  ```
-  Model Random Forest menunjukkan kinerja yang sangat baik dengan mendapatkan akurasi dan F1-score sempurna, tanpa ada kesalahan klasifikasi pada kelas mana pun.

---

#### **Logistic Regression**
- **Accuracy Score**: 0.12
- **F1-Score**: 0.06
- **Model Confusion Matrix**:
  ```
  [[ 25   0  85   0  21]
   [ 23   0 173   0  31]
   [ 25   0  69   0  31]
   [ 43   0 303   0  66]
   [ 25   0  59   0  21]]
  ```
-  Model Logistic Regression menunjukkan performa yang sangat buruk dengan akurasi dan F1-score yang sangat rendah. Confusion matrix menunjukkan banyak kesalahan klasifikasi, yang mengindikasikan bahwa model ini kesulitan membedakan kelas-kelas dengan baik.

---

#### **Support Vector Classifier (SVC)**
- **Accuracy Score**: 0.12
- **F1-Score**: 0.03
- **Model Confusion Matrix**:
  ```
  [[  0   0 123   0   8]
   [  0   0 219   0   8]
   [  0   0 118   0   7]
   [  0   0 399   0  13]
   [  0   0 102   0   3]]
  ```
- Model SVC juga menunjukkan performa yang buruk, mirip dengan Logistic Regression. Akurasi dan F1-score yang sangat rendah serta kesalahan klasifikasi yang signifikan terlihat pada confusion matrix.

---

### **Kesimpulan**
1. **Random Forest**: Model ini merupakan model dengan kinerja terbaik di antara yang lainnya.
2. **Logistic Regression dan SVC**: Kedua model ini mengalami kesulitan dalam klasifikasi, dengan akurasi dan F1-score yang rendah. Model-model ini tidak dapat menangkap pola yang ada dalam data dengan baik.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [1490]:
# Tuning Model
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [1493]:
# Prediksi menggunakan model terbaik
y_pred_tuned = best_model.predict(X_test)

accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
f1_tuned = f1_score(y_test, y_pred_tuned, average='weighted')
conf_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)

print(f'Accuracy setelah tuning: {accuracy_tuned:.2f}')
print(f'F1-Score setelah tuning: {f1_tuned:.2f}')
print('Confusion Matrix setelah tuning:')
print(conf_matrix_tuned)

Accuracy setelah tuning: 1.00
F1-Score setelah tuning: 1.00
Confusion Matrix setelah tuning:
[[131   0   0   0   0]
 [  0 227   0   0   0]
 [  0   0 125   0   0]
 [  0   0   0 412   0]
 [  0   0   0   0 105]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

In [1496]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_tuned))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       131
           1       1.00      1.00      1.00       227
           2       1.00      1.00      1.00       125
           3       1.00      1.00      1.00       412
           4       1.00      1.00      1.00       105

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



**Rekomendasi Tahapan Evaluasi Model:**

1. **Bandingkan Evaluasi Sebelum dan Setelah Tuning**:
   - Jika tuning belum dilakukan, lakukan dengan **GridSearchCV** atau **RandomizedSearchCV** untuk optimasi hyperparameter.

2. **Identifikasi Kelemahan Model**:
   - **Precision dan Recall**: Semua kelas memiliki hasil **1.00**, jadi tidak ada kelemahan teridentifikasi.
   - **Overfitting/Underfitting**: Jika hasil pada data uji sangat baik (100%), periksa apakah model overfit. Bandingkan kinerja di data pelatihan dan validasi.

3. **Rekomendasi Tindakan Lanjutan**:
   - **Jika Overfitting**: Tambahkan regularisasi, kurangi kompleksitas model, atau kumpulkan lebih banyak data.
   - **Jika Underfitting**: Gunakan model lebih kompleks, tambah fitur baru, atau perbaiki preprocessing data.
   - **Jika Model Belum Memuaskan**: Coba algoritma alternatif (XGBoost, LightGBM) atau metode ensemble (bagging, boosting).

4. **Evaluasi dengan Data Nyata**:
   - Uji model pada data nyata untuk memastikan kinerja dan hindari bias yang di sebabkan oleh class imbalance.